In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import os
import json
import matplotlib
import numpy as np

matplotlib.use("pdf")
os.chdir("/home/isevillano/Github/SHIELD/docs/")

/tmp/ipykernel_2128597/3152354622.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Comparativa Accuracy y Loss

In [2]:
df = pd.DataFrame(columns=['Dataset','Regularization','Percentage', 'Model','Test/Accuracy','Test/Loss'])

for dataset in os.listdir("../results/"):
    
    for experiment in os.listdir(f"../results/{dataset}/"):
        event_acc = EventAccumulator(f"../results/{dataset}/{experiment}/")
        config_file = json.load(open(f"../results/{dataset}/{experiment}/config.json"))
        event_acc.Reload()

        Test_Acc = None
        Test_Loss = None
        regularization = ""

        if "Test/Accuracy" in event_acc.Tags()["scalars"]:
            accs = [x.value for x in event_acc.Scalars("Test/Accuracy")]
            Test_Acc = max(accs)

        if "Test/Loss" in event_acc.Tags()["scalars"]:
            losses = [x.value if x.value != np.NAN else np.inf for x in event_acc.Scalars("Test/Loss")]
            Test_Loss = min(losses)

        if config_file["xshield"]:
            regularization = "X-SHIELD"
        if config_file["shield"]:
            regularization = "SHIELD"
        if regularization == "":
            regularization = "Baseline"
        df = pd.concat([df, pd.DataFrame({"Dataset": dataset, 
                                        "Regularization": regularization,
                                        "Model":config_file["pretrained_model"],
                                        "Test/Accuracy": Test_Acc,
                                        "Test/Loss": Test_Loss,
                                        "Percentage": config_file["percentage"]}, index=[0])], ignore_index=True)
df_show = df.copy()
# Mostrar las filas cuyo Test/Loss no sea un numero
df_show = df_show[df_show["Test/Loss"].apply(lambda x: np.isnan(x))]
display(df_show.sort_values(by="Dataset"))

/tmp/ipykernel_2128597/846789016.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,


/tmp/ipykernel_2128597/846789016.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_2128597/846789016.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_2128597/846789016.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when det

,Dataset,Regularization,Percentage,Model,Test/Accuracy,Test/Loss
60,EMNIST,X-SHIELD,9.0,efficientnet_v2_s,0.906064,NaN
62,EMNIST,X-SHIELD,12.0,efficientnet_v2_s,0.905319,NaN
91,OxfordIIITPet,SHIELD,3.0,efficientnet_v2_s,0.924775,NaN
93,OxfordIIITPet,SHIELD,6.0,efficientnet_v2_s,0.922867,NaN
97,OxfordIIITPet,SHIELD,12.0,efficientnet_v2_s,0.907877,NaN
100,OxfordIIITPet,SHIELD,3.0,efficientnet-b2,0.900518,NaN
104,OxfordIIITPet,SHIELD,9.0,efficientnet-b2,0.897520,NaN


In [3]:
df_latex = df.copy()
df_latex.rename(
    columns={
        "Dataset": "Dataset",
        "Percentage": "Percentage",
        "Model": "Model",
        "Test/Accuracy": "Accuracy",
        "Test/Loss": "Loss",
    },
    inplace=True,
)

df_latex = df_latex[df_latex["Regularization"].isin(["Baseline", "X-SHIELD", "SHIELD"])]
df_latex = df_latex[df_latex["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
df_latex["Accuracy"] = df_latex["Accuracy"].round(4)
df_latex["Loss"] = df_latex["Loss"].round(4)
df_latex = df_latex.sort_values(by=["Dataset", "Model", "Percentage", "Regularization"])

df_latex["Accuracy"] = (
    (df_latex["Accuracy"] * 100)
    .round(2)
    .map(lambda x: f"{x}\%")
    .map(lambda x: x.replace(".", ","))
)
df_latex["Loss"] = (
    df_latex["Loss"].round(4).map(lambda x: f"{x}").map(lambda x: x.replace(".", ","))
)

df_latex[["Accuracy", "Loss"]].to_csv("results.csv", index=False, sep=";")

df_latex["Percentage"] = df_latex["Percentage"].map(lambda x: f"{int(x)}\%")
df_latex = df_latex[
    ["Dataset", "Model", "Percentage", "Regularization", "Accuracy", "Loss"]
]
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("_", " "))
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("-", " "))

for dataset in df_latex["Dataset"].unique():
    df_ds = df_latex[df_latex["Dataset"] == dataset].drop(columns=["Dataset"])
    # df_ds = df_ds.drop(columns=["Model"])
    # Ordenar df_ds primero por modelo y luego por porcentaje

    df_ds.to_latex(
        f"tables/{dataset}_X-SHIELD.tex",
        index=False,
        escape=False,
        column_format="llccc",
    )

In [4]:
res = pd.read_csv("Experiments_mean.csv")
res = res[res["Regularization"].isin(["SHIELD", "X-SHIELD"])]
res = res[res["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
for metric in metrics:
    lista_figuras = []
    for j,percentage in enumerate(res["Percentage"].unique()):
        res_reg = res[res["Percentage"] == percentage]
        for k, regularization in enumerate(res["Regularization"].unique()): 
            df = res_reg[res_reg["Regularization"] == regularization]
            median = df[metric].quantile(0.5)
            RIQ = df[metric].quantile(0.75) - df[metric].quantile(0.25)
            df_show = df[(df[metric] > median - 1.5*RIQ) & (df[metric] < median + 1.5*RIQ)]
            figura = go.Violin(x = df_show["Model"]+"_"+df_show["dataset"],
                               y = df_show[metric],
                               name = f"{regularization} {percentage}%",
                               box_visible = True,
                               spanmode = "hard",
                               meanline_visible = True,
                               meanline_color = "black",
                               meanline_width = 2,
                               points = False)
            lista_figuras.append(figura)
    fig = go.Figure(data = lista_figuras)
    fig.update_layout(title = metric,
                    title_x = 0.5,
                    violinmode = "group")
    fig.show(renderer="svg")
    fig.write_image(f"./imagenes/{metric}_violin.pdf",width=1000,height=500)

,Dataset,Model,Percentage,Regularization,Accuracy,Loss
27,CIFAR10,efficientnet b2,0\%,Baseline,"97,64\%","0,0033"
28,CIFAR10,efficientnet b2,3\%,SHIELD,"97,6\%","0,0029"
29,CIFAR10,efficientnet b2,3\%,X-SHIELD,"97,59\%","0,0031"
30,CIFAR10,efficientnet b2,6\%,SHIELD,"97,4\%","0,0034"
31,CIFAR10,efficientnet b2,6\%,X-SHIELD,"97,53\%","0,0031"
...,...,...,...,...,...,...
94,OxfordIIITPet,efficientnet v2 s,6\%,X-SHIELD,"91,91\%","0,0089"
95,OxfordIIITPet,efficientnet v2 s,9\%,SHIELD,"91,88\%","0,0099"
96,OxfordIIITPet,efficientnet v2 s,9\%,X-SHIELD,"91,55\%","0,0104"
97,OxfordIIITPet,efficientnet v2 s,12\%,SHIELD,"90,79\%",nan


# Test Bayesiano SHIELD vs X-SHIELD

In [17]:
import numpy as np
import pandas as pd
import matplotlib

matplotlib.use("pdf")
import baycomp

df = pd.read_csv("Experiments_mean.csv")
df = df[df["Regularization"].isin(["SHIELD", "X-SHIELD"])]
#df = df[df["index"] < 100]


df_shield = df[df["Regularization"] == "SHIELD"]
df_xshield = df[df["Regularization"] == "X-SHIELD"]

# Nos quedamos con la intersección de los dataset donde "Model", "dataset", "Percentage" y "index" son iguales
df_instersection = df_shield.merge(df_xshield, on=["Model", "dataset", "Percentage", "index"], suffixes=["_shield", "_xshield"])


# Para cada métrica, comparamos los resultados de SHIELD y X-SHIELD con un test de Bayes con un rope igual al
# 5% de la diferencia de las medias
for percentage in df_instersection["Percentage"].unique():
    df_percentage = df_instersection[df_instersection["Percentage"] == percentage]
    for metric in metrics:
        rope = np.quantile(np.abs(df_percentage[metric+"_shield"] - df_percentage[metric+"_xshield"]), 0.20)
        SHIELD = df_percentage[metric+"_shield"]
        XSHIELD = df_percentage[metric+"_xshield"]
        bayes_test = baycomp.SignTest(SHIELD, XSHIELD,rope=rope,nsamples=100000)
        
        bayes_test.plot(names=["SHIELD", "XSHIELD"]).savefig(f"./imagenes/Bayes/{metric}_{percentage}.pdf", bbox_inches="tight", dpi=300,
                                                            transparent=True,
                                                            pad_inches=0,
                                                            )


    